In [1]:
!pip install transformers
!pip install pyarabic
!pip install farasapy

In [2]:
!git clone https://github.com/aub-mind/arabert

Cloning into 'arabert'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 208 (delta 117), reused 80 (delta 33), pack-reused 0
Receiving objects: 100% (208/208), 2.25 MiB | 5.70 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [2]:
from transformers import AutoModelWithLMHead, AutoTokenizer

modelv01 = AutoModelWithLMHead.from_pretrained('aubmindlab/bert-base-arabertv01')
tokenizerv01 = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabertv01')

modelv1 = AutoModelWithLMHead.from_pretrained('aubmindlab/bert-base-arabert')
tokenizerv1 = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')

model_safaya = AutoModelWithLMHead.from_pretrained('asafaya/bert-base-arabic')
tokenizersafaya = AutoTokenizer.from_pretrained('asafaya/bert-base-arabic')

model_mbert = AutoModelWithLMHead.from_pretrained('bert-base-multilingual-cased')
tokenizers_mbert = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

model_xlmr_large = AutoModelWithLMHead.from_pretrained('xlm-roberta-large')
tokenizers_xlmr_large = AutoTokenizer.from_pretrained('xlm-roberta-large')

model_xlmr_base = AutoModelWithLMHead.from_pretrained('xlm-roberta-base')
tokenizers_xlmr_base = AutoTokenizer.from_pretrained('xlm-roberta-base')

C:\Users\me250041\AppData\Local\Continuum\anaconda2\envs\py3.7\lib\site-packages\transformers\modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


OSError: Can't load config for 'aubmindlab/bert-base-arabertv01'. Make sure that:

- 'aubmindlab/bert-base-arabertv01' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'aubmindlab/bert-base-arabertv01' is the correct path to a directory containing a config.json file



In [14]:
from transformers import pipeline

pipeline_fill_mask_arabertv01 = pipeline("fill-mask",model=modelv01, tokenizer=tokenizerv01)
pipeline_fill_mask_arabertv1 = pipeline("fill-mask",model=modelv1, tokenizer=tokenizerv1)
pipeline_fill_mask_asafaya = pipeline("fill-mask",model=model_safaya, tokenizer=tokenizersafaya)

pipeline_fill_mask_mbert = pipeline("fill-mask",model=model_mbert, tokenizer=tokenizers_mbert)
pipeline_fill_mask_xlmr_large = pipeline("fill-mask",model=model_xlmr_large, tokenizer=tokenizers_xlmr_large)
pipeline_fill_mask_xlmr_base = pipeline("fill-mask",model=model_xlmr_base, tokenizer=tokenizers_xlmr_base)

In [15]:
from pprint import pprint

In [8]:
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess_arabert import preprocess

farasa_segmenter = FarasaSegmenter(interactive=True)

perform system check...
check java version...
Your java version is 11.0 which is compatiple with Farasa 
check toolkit binaries...
Dependencies seem to be satisfied..
initializing [SEGMENT] task in INTERACTIVE mode...


/usr/local/lib/python3.6/dist-packages/farasa/__base.py:45: UserWarning: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
  "Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases."


task [SEGMENT] is initialized interactively.


In [11]:
text = " عاصمة لبنان هي [MASK]"
textv1 = preprocess(text, do_farasa_tokenization=True , farasa = farasa_segmenter, use_farasapy = True)
textv1 = textv1.replace("MASK","[MASK]")
textv1 = textv1.replace(":[",":")
text_xlm_r = text.replace("[MASK]","<mask>")
pprint(text.split())
print("#---------------------------------#")
print(textv1)
print("#---------------------------------#")
print(text_xlm_r)

['عاصمة', 'لبنان', 'هي', '[MASK]']
#---------------------------------#
عاصم +ة لبنان هي [MASK]
#---------------------------------#
 عاصمة لبنان هي <mask>


In [18]:
#check if the intended word is 1 subword
print(tokenizerv01.tokenize('بيروت'))
print(tokenizerv1.tokenize('بيروت'))
print(tokenizersafaya.tokenize('بيروت'))
print(tokenizers_mbert.tokenize('بيروت'))
print(tokenizers_xlmr_large.tokenize('بيروت'))
print(tokenizers_xlmr_base.tokenize('بيروت'))

['بيروت']
['بيروت']
['بيروت']
['بيروت']
['▁بيروت']
['▁بيروت']


In [19]:
print("MASK to predict: ",text)
print("#-----------AraBERTv0.1-------------#")
pprint(pipeline_fill_mask_arabertv01(text,))
print("#-----------AraBERTv1------------#")
pprint(pipeline_fill_mask_arabertv1(textv1))
print("#-----------asafaya_BERT--------------#")
pprint(pipeline_fill_mask_asafaya(text))
print("#-----------mBERT-------------#")
pprint(pipeline_fill_mask_mbert(text))
print("#-----------XLM-R_large------------#")
pprint(pipeline_fill_mask_xlmr_large(text_xlm_r))
print("#-----------XLM-R_base--------------#")
pprint(pipeline_fill_mask_xlmr_base(text_xlm_r))

MASK to predict:   عاصمة لبنان هي [MASK]
#-----------AraBERTv0.1-------------#
[{'score': 0.16055037081241608,
  'sequence': '[CLS] عاصمة لبنان هي طرابلس [SEP]',
  'token': 37024,
  'token_str': 'طرابلس'},
 {'score': 0.1498166173696518,
  'sequence': '[CLS] عاصمة لبنان هي بيروت [SEP]',
  'token': 20456,
  'token_str': 'بيروت'},
 {'score': 0.05173841491341591,
  'sequence': '[CLS] عاصمة لبنان هي بغداد [SEP]',
  'token': 20202,
  'token_str': 'بغداد'},
 {'score': 0.03738606348633766,
  'sequence': '[CLS] عاصمة لبنان هي دمشق [SEP]',
  'token': 8369,
  'token_str': 'دمشق'},
 {'score': 0.034828972071409225,
  'sequence': '[CLS] عاصمة لبنان هي. [SEP]',
  'token': 17031,
  'token_str': '.'}]
#-----------AraBERTv1------------#
[{'score': 0.6333718299865723,
  'sequence': '[CLS] عاصم +ة لبنان هي بيروت [SEP]',
  'token': 32803,
  'token_str': 'بيروت'},
 {'score': 0.16333439946174622,
  'sequence': '[CLS] عاصم +ة لبنان هي. [SEP]',
  'token': 11,
  'token_str': '.'},
 {'score': 0.09127318859100342

In [ ]:
#original word was عادل
text_3 = "أكد نائب رئيس مجلس الوزراء السابق غسان حاصباني أن المحاسبة ال+ [MASK] +ة والشفافة والتي تطاول الجميع اساسية، ولتكن الكلمة الفصل للقضاء، مشيرا الى ان على ما يبدو السياسة متداخلة في الكثير من الملفات، ومتمنيا أن يتوقف ذلك."
text_3 = preprocess(text3, do_farasa_tokenization=True , farasa = farasa_segmenter, use_farasapy = True)
text_3 = text_3.replace("MASK","[MASK]")
pprint(text_3.split())

['أكد',
 'نائب',
 'رئيس',
 'مجلس',
 'ال+',
 'وزراء',
 'ال+',
 'سابق',
 'غسان',
 'حاصباني',
 'أن',
 'ال+',
 'محاسب',
 '+ة',
 'ال+',
 '[MASK]',
 '+ة',
 'و+',
 'ال+',
 'شفاف',
 '+ة',
 'و+',
 'التي',
 'تطاول',
 'ال+',
 'جميع',
 'اساسي',
 '+ة',
 '،',
 'و+',
 'ل+',
 '+ت',
 '+كن',
 'ال+',
 'كلم',
 '+ة',
 'ال+',
 'فصل',
 'ل+',
 'ال+',
 'قضاء',
 '،',
 'مشير',
 '+ا',
 'الى',
 '+ان',
 'على',
 'ما',
 'يبدو',
 'ال+',
 'سياس',
 '+ة',
 'متداخل',
 '+ة',
 'في',
 'ال+',
 'كثير',
 'من',
 'ال+',
 'ملف',
 '+ات',
 '،',
 'و+',
 'متمني',
 '+ا',
 'أن',
 'يتوقف',
 'ذلك',
 '.']


In [ ]:
pipeline_fill_mask_arabertv1(text_3)

[{'score': 0.351762056350708,
  'sequence': '[CLS] أكد نائب رئيس مجلس ال+ وزراء ال+ سابق غسان حاصباني أن ال+ محاسب +ة ال+ عادل +ة و+ ال+ شفاف +ة و+ التي تطاول ال+ جميع اساسي +ة ، و+ ل+ +ت +كن ال+ كلم +ة ال+ فصل ل+ ال+ قضاء ، مشير +ا الى +ان على ما يبدو ال+ سياس +ة متداخل +ة في ال+ كثير من ال+ ملف +ات ، و+ متمني +ا أن يتوقف ذلك. [SEP]',
  'token': 18922},
 {'score': 0.18136297166347504,
  'sequence': '[CLS] أكد نائب رئيس مجلس ال+ وزراء ال+ سابق غسان حاصباني أن ال+ محاسب +ة ال+ حقيقي +ة و+ ال+ شفاف +ة و+ التي تطاول ال+ جميع اساسي +ة ، و+ ل+ +ت +كن ال+ كلم +ة ال+ فصل ل+ ال+ قضاء ، مشير +ا الى +ان على ما يبدو ال+ سياس +ة متداخل +ة في ال+ كثير من ال+ ملف +ات ، و+ متمني +ا أن يتوقف ذلك. [SEP]',
  'token': 34985},
 {'score': 0.05650452524423599,
  'sequence': '[CLS] أكد نائب رئيس مجلس ال+ وزراء ال+ سابق غسان حاصباني أن ال+ محاسب +ة ال+ نزيه +ة و+ ال+ شفاف +ة و+ التي تطاول ال+ جميع اساسي +ة ، و+ ل+ +ت +كن ال+ كلم +ة ال+ فصل ل+ ال+ قضاء ، مشير +ا الى +ان على ما يبدو ال+ سياس +ة متداخل +ة في ال+